In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten, Input
from tensorflow.keras.datasets import mnist
from keras import Sequential

# Model creation

In [34]:
class ObjectDetection(tf.keras.Model):
    def __init__(self,n_class):
        super(ObjectDetection,self).__init__()
        self.n_class=n_class
        self.base_model=Sequential([
            Conv2D(filters=64,kernel_size=3,padding='same',strides=1,activation='relu'),
            BatchNormalization(),
            MaxPool2D(pool_size=2,strides=2),
            Conv2D(filters=128,kernel_size=3,padding='same',strides=1,activation='relu'),
            BatchNormalization(),
            Conv2D(filters=128,kernel_size=3,padding='same',strides=1,activation='relu'),
            BatchNormalization()
        ])

        self.regressor = Sequential([
			Linear(baseModel.fc.in_features, 128),
			ReLU(),
			Linear(128, 64),
			ReLU(),
			Linear(64, 32),
			ReLU(),
			Linear(32, 4),
			Sigmoid()]
		)

    def call(self,inputs):
        x=self.base_model(inputs)



        return x

In [62]:
input_tensor = Input(shape=(224, 224, 3))
model = ObjectDetection(n_class=n_class)
output=model(input_tensor)

#model.base_model.summary()

model.base_model.summary()


Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_105 (Conv2D)         (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization_105 (B  (None, 224, 224, 64)      256       
 atchNormalization)                                              
                                                                 
 max_pooling2d_35 (MaxPooli  (None, 112, 112, 64)      0         
 ng2D)                                                           
                                                                 
 conv2d_106 (Conv2D)         (None, 112, 112, 128)     73856     
                                                                 
 batch_normalization_106 (B  (None, 112, 112, 128)     512       
 atchNormalization)                                              
                                                     

In [52]:
for layer in model.layers:
    print(layer.__class__.__name__)
    print(layer.get_config())
    print("--------------------------------------------------")

Sequential
{'name': 'sequential_26', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 224, 224, 3), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'conv2d_78_input'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'conv2d_78', 'trainable': True, 'dtype': 'float32', 'filters': 64, 'kernel_size': (3, 3), 'strides': (1, 1), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}, 'registered_name': None, 'build_confi

In [61]:
n_class = 10
input_tensor = Input(shape=(224, 224, 3))
model = ObjectDetection(n_class=n_class)
output=model(input_tensor)

model=tf.keras.Model(inputs=input_tensor,outputs=output)

model.base_model.summary()

AttributeError: 'Functional' object has no attribute 'base_model'